In [1]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Set up for reproducible results
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

2023-12-19 18:40:28.360014: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 18:40:28.360117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 18:40:28.397504: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 18:40:28.484017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 18:40:29.977250: W tensorflow/compiler/tf2

In [2]:
# Paths to the .npy files
categories_path = 'training_dataset/categories.npy'
valid_periods_path = 'training_dataset/valid_periods.npy'
training_data_path = 'training_dataset/training_data.npy'

# Load the data
categories = np.load(categories_path)
valid_periods = np.load(valid_periods_path)
training_data = np.load(training_data_path)

In [3]:
# Constants for sequence and forecast lengths
seq_length = 128
forecast_length = 9

# Initialize two scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [4]:
# Function to preprocess data
def preprocess_data(data, valid_periods, seq_length, forecast_length, scaler_X, scaler_y):
    X, y = [], []
    for i, row in enumerate(data):
        valid_data = row[valid_periods[i][0]:valid_periods[i][1]]
        if valid_data.size > 0:
            for j in range(len(valid_data) - seq_length - forecast_length + 1):
                seq_X = valid_data[j:(j + seq_length)]
                seq_y = valid_data[(j + seq_length):(j + seq_length + forecast_length)]
                
                X.append(seq_X)
                y.append(seq_y)
    
    X = scaler_X.fit_transform(np.array(X))
    y = scaler_y.fit_transform(np.array(y))

    return np.array(X), np.array(y)

In [5]:
# Preprocess the data
X, y = preprocess_data(training_data, valid_periods, seq_length, forecast_length, scaler_X, scaler_y)

In [6]:
# Save the fitted scalers
scaler_X_filename = "Exp1/scaler_X.save"
scaler_y_filename = "Exp1/scaler_y.save"
dump(scaler_X, scaler_X_filename)
dump(scaler_y, scaler_y_filename)

['Exp1/scaler_y.save']

In [7]:
# Split the data into training and validation sets
val_size = int(len(X) * 0.2)
indices = np.arange(len(X))
np.random.shuffle(indices)
X_train, y_train = X[:-val_size], y[:-val_size]
X_val, y_val = X[-val_size:], y[-val_size:]

In [8]:
# Reshape data for LSTM input
X_train = X_train.reshape((X_train.shape[0], seq_length, 1))
y_train = y_train.reshape((y_train.shape[0], forecast_length, 1))
X_val = X_val.reshape((X_val.shape[0], seq_length, 1))
y_val = y_val.reshape((y_val.shape[0], forecast_length, 1))

In [9]:
# Model building function
def build_model(input_shape, lstm_units, dropout_rate, forecast_length):
    input_layer = tfkl.Input(shape=input_shape)
    x = tfkl.Bidirectional(tfkl.LSTM(lstm_units, return_sequences=True))(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    x = tfkl.LSTM(lstm_units // 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(dropout_rate)(x)
    output_layer = tfkl.Dense(forecast_length, activation='linear')(x)  # Linear activation for regression

    model = tfk.Model(inputs=input_layer, outputs=output_layer)
    return model

# Model configuration
input_shape = (seq_length, 1)
dropout_rate = 0.2
lstm_units = 128

In [10]:
# Build the model
model = build_model(input_shape, lstm_units, dropout_rate, forecast_length)

2023-12-19 18:40:50.891866: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:40:50.996635: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:40:50.996685: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:40:50.998627: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-19 18:40:50.998672: I external/local_xla/xla/stream_executor

In [11]:
# Compile the model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='mse')

In [12]:
# Train the model
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train,
    y_train,
    batch_size=128,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

2023-12-19 18:40:54.523935: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1728679424 exceeds 10% of free system memory.
2023-12-19 18:40:55.726419: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1728679424 exceeds 10% of free system memory.


Epoch 1/100


2023-12-19 18:41:03.322878: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-19 18:41:04.381657: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f17081493c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-19 18:41:04.381691: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2023-12-19 18:41:04.391484: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703007664.640506     555 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26378/26378 [==============================] - 643s 24ms/step - loss: 0.1320 - val_loss: 0.1364
Epoch 2/100
26378/26378 [==============================] - 613s 23ms/step - loss: 0.1263 - val_loss: 0.1327
Epoch 3/100
26378/26378 [==============================] - 610s 23ms/step - loss: 0.1202 - val_loss: 0.1152
Epoch 4/100
26378/26378 [==============================] - 606s 23ms/step - loss: 0.1108 - val_loss: 0.1101
Epoch 5/100
26378/26378 [==============================] - 599s 23ms/step - loss: 0.1072 - val_loss: 0.1089
Epoch 6/100
26378/26378 [==============================] - 600s 23ms/step - loss: 0.1040 - val_loss: 0.1086
Epoch 7/100
26378/26378 [==============================] - 598s 23ms/step - loss: 0.1025 - val_loss: 0.1081
Epoch 8/100
26378/26378 [==============================] - 597s 23ms/step - loss: 0.1011 - val_loss: 0.1073
Epoch 9/100
26378/26378 [==============================] - 595s 23ms/step - loss: 0.0991 - val_loss: 0.1062
Epoch 10/100
26378/26378 [==============

In [13]:
# Evaluate the model on the validation set (or test set if available)
evaluation_results = model.evaluate(X_val, y_val)
print(f'Validation Loss: {evaluation_results}')

# Save the model
model_save_path = 'Exp1/model'
model.save(model_save_path)

26378/26378 [==============================] - 214s 8ms/step - loss: 0.1055
Validation Loss: 0.10550430417060852
INFO:tensorflow:Assets written to: Exp1/model/assets


INFO:tensorflow:Assets written to: Exp1/model/assets


In [14]:
# Evaluate on original (unscaled) validation data 
# note: there could be bias here, since the scaler was fit on all data; distribution properties might have spread
y_val_org = scaler_y.inverse_transform(y_val.reshape((-1, forecast_length)))
y_pred = model.predict(X_val)
y_pred_iscaled = scaler_y.inverse_transform(y_pred.reshape((-1, forecast_length)))
mse = tfk.losses.MeanSquaredError()
print(f"Val loss (MSE): {mse(y_val_org, y_pred_iscaled).numpy()}")

# Val loss for each prediction step
for t in range(forecast_length):
    mse = tfk.metrics.MeanSquaredError()
    mse.update_state(y_val_org[:, t], y_pred_iscaled[:, t])
    print(f'Val loss (MSE) {t+1} step forward: {mse.result().numpy()}')

26378/26378 [==============================] - 195s 7ms/step
Val loss (MSE): 0.00677393889054656
Val loss (MSE) 1 step forward: 0.003047530073672533
Val loss (MSE) 2 step forward: 0.004136371426284313
Val loss (MSE) 3 step forward: 0.0051204063929617405
Val loss (MSE) 4 step forward: 0.006041180342435837
Val loss (MSE) 5 step forward: 0.006898761726915836
Val loss (MSE) 6 step forward: 0.007714167237281799
Val loss (MSE) 7 step forward: 0.008557872846722603
Val loss (MSE) 8 step forward: 0.009331969544291496
Val loss (MSE) 9 step forward: 0.010117187164723873
